In [ ]:
import os
import nbformat #for running jupyter notebooks
import nbparameterise as nbparam #for changing input parameters in jupyter notebooks
from datetime import date
from constants import MTF_PATH

In [ ]:
#set the names of mtf_visualize notebooks to use
MTF_VISUALIZE_NAMES = ['mtf_visualize_time']

#set the names of the scout folders you want to visualize
SCOUT_FOLDER_NAMES = ['Main 2 - LobbyCS']

In [ ]:
os.listdir(MTF_PATH) 
#take a look at which scouts we'll be visualizing
#ideally there are no folders in this directory that aren't scout folders

In [ ]:
scout_folder_paths = []
for name in SCOUT_FOLDER_NAMES:
    assert(name in os.listdir(MTF_PATH))
    scout_folder_paths.append(MTF_PATH + os.sep + name)

In [ ]:
#take a look at the aviailable visualization notebooks
this_directory = os.listdir()
this_directory

In [ ]:
#all of the visualization notebooks specified in constants.py should exist in the same directory as this one
for name in MTF_VISUALIZE_NAMES: assert(name + ".ipynb" in this_directory)

For each selected mtf_visualize notbook, run that notebook on all scouts.

Visualizations will be saved as images in the scout's folder. 

Folders will be created (in the same location as main.ipynb) containing executed versions of each notebook.  Thier names will be of the form:
* [notebook name]\_executed

In [ ]:
for notebook_name in MTF_VISUALIZE_NAMES:
    
    #make directory for executed version
    mtf_visualize_executed_dir = notebook_name + "_executed"
    try:
        os.mkdir(mtf_visualize_executed_dir)
    except FileExistsError:
        pass #dir already exists, do nothing
    
    for scout_folder_path in scout_folder_paths:
        
        #use nbformat to read mtf_visualize file into a notebook object
        with open(notebook_name + ".ipynb") as f:
            nb = nbformat.read(f, as_version=4)
    
        #replace SCOUT_FOLDER_PATH parameter in first cell of notebook we want to run
        orig_params = nbparam.extract_parameters(nb)
        new_params = nbparam.parameter_values(orig_params, SCOUT_FOLDER_PATH = scout_folder_path)
        new_nb = nbparam.replace_definitions(nb, new_params)
        #new notebook gets executed automatically
    
        #save the new notebook, executed with replaced params.
        executed_notebook_name = notebook_name + "_" + os.path.basename(scout_folder_path) + ".ipynb"
        with open(mtf_visualize_executed_dir + os.sep + executed_notebook_name, 'w') as f:
            nbformat.write(new_nb, f)
 